# Évaluation du modèle DQN entraîné

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def evaluate_agent(env, agent, n_episodes=50):
    """
    Évalue les performances de l'agent sur n_episodes sans exploration.
    """
    print(f"--- Démarrage de l'évaluation sur {n_episodes} épisodes ---")
    
    # 1. Sauvegarder l'epsilon actuel et le mettre à 0 pour le test (Exploitation pure)
    original_epsilon = agent.epsilon
    agent.epsilon = 0.0
    
    # Mettre le réseau en mode évaluation (pas de calcul de gradients inutile)
    agent.q_network.eval()
    
    game_scores = []      # Vrais scores du jeu (points affichés à l'écran)
    total_rewards = []    # Rewards cumulées (ce que l'IA optimise)
    steps_survived = []   # Durée de survie

    for i in range(n_episodes):
        state = env.reset()
        done = False
        episode_reward = 0
        episode_steps = 0
        
        while not done:
            # L'agent choisit l'action (sans aléatoire car epsilon=0)
            action = agent.act(state)
            next_state, reward, done = env.step(action)
            
            state = next_state
            episode_reward += reward
            episode_steps += 1
            
        # Fin de l'épisode
        game_scores.append(env.game.score) # On récupère le score interne du jeu
        total_rewards.append(episode_reward)
        steps_survived.append(episode_steps)
        
        if (i+1) % 10 == 0:
            print(f"Episode {i+1}/{n_episodes} - Score Jeu: {env.game.score} - Reward: {episode_reward:.2f}")

    # 2. Restaurer la configuration de l'agent
    agent.epsilon = original_epsilon
    agent.q_network.train() # Remettre en mode train si besoin
    
    # 3. Calcul des statistiques
    avg_score = np.mean(game_scores)
    std_score = np.std(game_scores)
    max_score = np.max(game_scores)
    
    avg_reward = np.mean(total_rewards)
    avg_steps = np.mean(steps_survived)
    
    print("\n=== RÉSULTATS DE L'ÉVALUATION ===")
    print(f"Score Moyen (Jeu)   : {avg_score:.2f} ± {std_score:.2f}")
    print(f"Score Max (Jeu)     : {max_score}")
    print(f"Reward Moyenne (RL) : {avg_reward:.2f}")
    print(f"Survie Moyenne      : {avg_steps:.1f} steps")
    
    return game_scores, total_rewards

# --- Exécution de l'évaluation ---
# Assurez-vous que 'env' et 'agent' sont bien instanciés (ceux de votre fin d'entraînement)
test_scores, test_rewards = evaluate_agent(env, agent, n_episodes=50)

# --- Visualisation Graphique ---
plt.figure(figsize=(12, 5))

# 1. Histogramme des scores
plt.subplot(1, 2, 1)
plt.hist(test_scores, bins=10, color='skyblue', edgecolor='black')
plt.title('Distribution des Scores de Jeu')
plt.xlabel('Score')
plt.ylabel('Nombre d\'épisodes')
plt.axvline(np.mean(test_scores), color='red', linestyle='dashed', linewidth=1, label=f'Moyenne: {np.mean(test_scores):.1f}')
plt.legend()

# 2. Boxplot (pour voir la stabilité)
plt.subplot(1, 2, 2)
plt.boxplot(test_scores)
plt.title('Dispersion des Scores (Stabilité)')
plt.ylabel('Score')
plt.xticks([1], ['Agent DQN'])

plt.tight_layout()
plt.show()